# Capstone Proposals:
    
As Americans, we have an enormous variety of foods commonly available at grocery stores, corner shops, and increasingly, online. For one project, I'd like to take a look at the fundamental ingredients and nutrients that compose this great variety. My hope would be to predict the number of nutrients found in certain food items, as well as looking hollistically at the ingredients that are most common in our foods. I would like to then relate this to general dietary habits of the US population, and get a granular idea of the nutrient composition of diets. The stretch goal would be to predict the changes in nutrient-use given an increasing population of vegans, where I could estimate increases in required nutrient needs based on 'normal' diets. I'm unsure whether this would be kosher, because obviously I can't check the quality of my predictions for the future. I could, however, see if I could predict correlations on historical data in growth of the vegan diet in the US, and then use that model to predict the effect of future growth of vegan diets.
       
*** (I'm still pondering the best way to go about this, but at least you can see where I'm hoping to take this) ***
    
'Nutrient Database from 2012' - https://catalog.data.gov/dataset/usda-national-nutrient-database-for-standard-reference
'Nutrient Database from 2009' - https://catalog.data.gov/dataset/usda-national-nutrient-database-for-standard-reference-release-22
    
Partially pre-cleaned nutrient data - https://github.com/mhess126/usda_national_nutrients
USDA API (not sure if this could be useful yet?) - https://ndb.nal.usda.gov/ndb/api/doc
    
    
I'm still looking for better data to work with that could give me some idea of dietary habits, but here's where I've been looking - https://catalog.data.gov/dataset?q=bureauCode:%22005:13%22 ; https://catalog.data.gov/dataset?q=usda+consumption+national+nutrient&sort=views_recent+desc&ext_location=&ext_bbox=&ext_prev_extent=-142.03125%2C2.4601811810210052%2C-59.0625%2C58.63121664342478
    
    
Alternatively, I could look at pricing these ingredients, based on the foodtypes that we find them in. For example, take a chili sauce. Of this sauce, take a look at the unique ingredients, and their respective portion size in the sauce. Let's say that black beans compose 20% of the chili, and the chili runs 5 dollars/unit. Then the pricing for the black bean ingredient would be 1 dollar.

From there, I would want to observe how expensive these ingredients can get and how their prices change depending on what products they may be found in. 
    
'Food Price Outlook, current' - https://catalog.data.gov/dataset/food-price-outlook
    

In [1]:
# pip install scrapy
# pip install --upgrade zope2

import foursquare
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import requests
from scrapy import Selector
from scrapy.http import HtmlResponse
import seaborn as sns
import time
import unicodedata

In [2]:
CLIENT_ID = '33NDJLQ342FAMTNX5Z55PR0PQQOJZRAZZ3XEAI0ERQXEJRUL'
CLIENT_SECRET = 'FGMFNZGMWUR1ILZFGH2NV1OKQY3WK5AAPHWKXTFWRR3B4Z4E'
client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

In [20]:
#Let's define a geo dictionary whose bounds encompass all of SF, and another 4 miles south as well (so ~7mi x 11mi)
ne = {'ne_lat': -122.3550796509, 'ne_long': 37.8127675576}
sw = {'sw_lat': -122.5164413452, 'sw_long': 37.7078622611}

# east/west
lat_bounds = [ne['ne_lat'], sw['sw_lat']]
print lat_bounds[0]
# north/south
lon_bounds = [ne['ne_long'], sw['sw_long']]
print lon_bounds

#increment ~ half a mile (in latitude/longitude)
increment = 0.017

# The gridding below moves North, starting from the bottom SW corner boundary, and then moves east half a mile,
# and repeats the process until stopping at the NE corner boundary.
grid_pairs = []
for lat in np.arange(lat_bounds[1], lat_bounds[0], increment):
    for lon in np.arange(lon_bounds[1], lon_bounds[0], increment):
        grid_pairs.append([lat, lon])
        
print len(grid_pairs)
# for x, y in grid_pairs:
#     print x, y

-122.355079651
[37.8127675576, 37.7078622611]
70


In [4]:
# This will independently pull the venue names and id codes that correspond to the geographical areas
# I paired off in the previous step with grid_pairs. The names and id's will be subesquently used to
# construct menu url's, which I then intend to scrape.
unique_venues_from_search = []
unique_venue_names_from_search = []
start_time = time.time()

#My hypothesis is that there are 18k items to get (360 pairs * 50 offset limit)
urls_collected = 0
# for i in range(len(grid_pairs)+1):
#     single_loop_limit = i * 499.
#     print "loop # %d " %i
for x, y in grid_pairs:
    try:
        search = client.venues.search(params={'ll': '%.2f, %.2f' % (y, x),'query': 'food', 'limit':'50',
                                      'intent':'browse','radius':'800'})
        searched_venue_ids = [search['venues'][i]['id'] for i in range(len(search['venues']))]
        searched_name_ids = [search['venues'][i]['name'] for i in range(len(search['venues']))]
        for next_id, next_name in zip(searched_venue_ids, searched_name_ids):
            unique_venues_from_search.append(next_id)
            unique_venue_names_from_search.append(next_name)
#         print('--- %s loop-active seconds ---' % (time.time() - start_time))
    except:
        print('Sleeping...')
#         time.sleep(random.randint(115,140))
print('--- %s active seconds ---' % (time.time() - start_time))

--- 11.4930729866 active seconds ---


In [5]:
print len(unique_venue_names_from_search), len(unique_venues_from_search)

358 358


In [6]:
unique_venue_names_from_search[:5], unique_venues_from_search[:5]

([u'Food Fair',
  u'Food Fair',
  u'Asian American Food Company',
  u'Other Avenues Food Store',
  u'7-Eleven'],
 [u'4e3d8765ae60454236667cc4',
  u'4e3d8765ae60454236667cc4',
  u'463bfdccf964a52026461fe3',
  u'4a90954ff964a520a61820e3',
  u'4afba001f964a520d51e22e3'])

In [7]:
#NOTE: THIS ISN'T WORKING B/C SOME OF THE MENUS DON'T ALIGN PROPERLY IN MY FOOR LOOP - ASK DAVIDDD

#this works for the .search api calls only (need a diff process b/c .explore doesn't return names)
menu_urls_from_search = []
base_url = 'https://foursquare.com/v/'
for venue_id, venue_name in zip(unique_venues_from_search, unique_venue_names_from_search):
    dat_id = unicodedata.normalize('NFKD', venue_id).encode('ascii','ignore')
    dat_name = unicodedata.normalize('NFKD', venue_name).encode('ascii','ignore')
    dat_name = dat_name.lower().replace('/','-').replace(' ','-')
    transformed_url = base_url+dat_name+'/'+dat_id+'/menu'
    menu_urls_from_search.append(transformed_url)
len(menu_urls_from_search)

358

In [8]:
# Testing that no venues have been duplicated with my searches
unique_urls_from_search = list(set(menu_urls_from_search))

In [ ]:
#NOTE: HERE IS WHERE I LAST HAD MY .EXPLORE FUNCTIONALITY

In [31]:
unique_venues = []
for offset in range(50, 200, 50):
    #intent:browse let's use use radius; radius is radius in meters around given 'll'; 800 meters (dimension I elected)
    #is approx 0.5 miles (but I may adjust the radius going forward)
    #note: i don't need to include the radius here, b/c foursquare will automatically modify it dependent on
    #the density of venues in that area, but just to be sure i grab as many as possible...
    
    search = client.venues.explore(params={'ll': '%.2f, %.2f' % [(y, x) for x, y in grid_pairs], 'radius': '800',
                                          'section': 'food','limit':'50','offset':''+str(offset)+''})
    
    #NOTE: i need to figure out what the response variables are for .explore, so i know how to grab the venue_id's
    #NOTE: no offset available in the .search call
    
    #if i use this (below) api call, i need to pull out all the non-food places (which won't have menus)
    #i'd still have to do that regularly, but this could mean i get way fewer venues of value (read: food venues)
    #alternatively, by using the above api call, i can guarantee i pull food items, but only the most popular ones.
    #so either way, there might be some missing info. If i collate all the data though, and put those venues together,
    #i should get a pretty comprehensive list!

    
    
    #... so offset won't work for this..., only for .explore
    ven_ids[offset] = [search['venues'][i]['id'] for i in range(len(search['venues']))]
    

In [25]:
# Now that I have plenty of urls, let's plug them into a scraper so i can populate my df. Here are the headers
# I'm looking to get as well...

column_headers = ['venue_name', 'venue_desc_list', 'venue_menu_url', 'venue_rated', 'meta_menu_n', 'depth_menus_n',
                  'menu_item_name', 'menu_item_price', 'menu_item_desc']

df_ready_rows = []
def parse_url(url, data=False):
    
#     venue_rows = []
    response               =  requests.get(url)
    
    #Steps:
    #1) get the unicode objects
    #2) change objects from unicode to string
    
    venue_name_uni         =  Selector(text=response.text).xpath('//h1[@class="venueName"]/text()').extract()
    venue_name             = unicodedata.normalize('NFKD', venue_name_uni[0]).encode('ascii','ignore')
    
    #also need to do an iteration to capture the multiple descriptors
    venue_desc_uni         =  Selector(text=response.text).xpath('//span[@class="unlinkedCategory"]/\
    text()').extract()
    
    #I'll make a list to capture the entire description:
    venue_desc_list        = []
    for venue_desc_phrase in range(len(venue_desc_uni)):
        venue_desc_n       = unicodedata.normalize('NFKD', venue_desc_uni[venue_desc_phrase]).encode('ascii',
                                                                                                  'ignore')
        venue_desc_list.append(venue_desc_n)

    #The url too, right?
    venue_menu_url         = url

    #Grabbing the venue rating, just a note: venueScore positive/neutral/negative, but I'm only getting the
    #rating 1-10
    venueScore_options     = ['positive','neutral','negative']
    venue_rated = []
    for vs_option in venueScore_options:
        try:
            venue_rating_uni        = Selector(text=response.text).xpath('//div[@class="venueRateBlock  "]/\
    span[@class="venueScore '+vs_option+'"]/span/text()').extract()
            venue_rated             = unicodedata.normalize('NFKD', venue_rating_uni[0]).encode('ascii','ignore')
            venue_rated = float(venue_rated)
        except:
            pass
        
    #Even if there is no rating, I'd still like to keep track of that...
    if venue_rated == np.nan:
        venue_rated = 'rating_not_available'
        
    #And I'll transform the list back into a string...
#     venue_rated = venue_rated[0]
    
    #NOTE: do i also need to account for when menus don't have titles? because in that case meta_menu_list
    #could/would
    #return null. if so, perhaps just do a 'try excepct:pass' function if it can't find titles, but could it still
    #grab the menu items? maybe i should just put in a "null title" for the meta_menu_n to overcome this
    #I no longer think this is an issue, but maybe something to put in the appendix for later?
    
    meta_menu_list      =  Selector(text=response.text).xpath('//h2[@class="categoryName"]/text()').extract()
        
    for meta_menu_item in range(len(meta_menu_list)):
        
        meta_menu_n         = unicodedata.normalize('NFKD', meta_menu_list[meta_menu_item]).encode('ascii',
                                                                                                   'ignore')
        
#         print "meta menu title %d:" %(meta_menu_item+1), meta_menu_n, "# of meta menus:", len(meta_menu_list)
        
        depth_menus_n_uni = Selector(text=response.text).xpath('//div[@class="menu"]['+str(meta_menu_item+1)+']/\
        div[@class="menuItems"]/div[@class="section"]/div[@class="sectionHeader"]/\
        div[@class="sectionName"]/text()').extract()
        
        for meta_depth_nn in range(len(depth_menus_n_uni)):
            
            depth_menus_n     = unicodedata.normalize('NFKD', depth_menus_n_uni[meta_depth_nn]
                                                     ).encode('ascii','ignore')

            #get the name of the depth menu, and record it's location as 'n_level'
            n_level = meta_depth_nn+1
#             print "depth menu title %d:" %(n_level), depth_menus_n
            
            #let's grab the entire depth menu:
            depth_menu_id_uni = Selector(text=response.text).xpath('//div[@class="menu"]\
            ['+str(meta_menu_item+1)+']/div[@class="menuItems"]/div[@class="section"]['+str(n_level)+']/\
            div[@class="sectionHeader"]/div[@class="sectionName"]/text()').extract()
            depth_menu_id     = unicodedata.normalize('NFKD', depth_menu_id_uni[0]
                                                     ).encode('ascii','ignore')
            depth_menu_id = len(depth_menu_id_uni)
#             print "#id of depth menu:", depth_menu_id
            
            #loop throught the left and right side of each container:
            left_or_right_list = ['left','right']
            
            for left_or_right in left_or_right_list:
    
                #need the length of the [left/right] container, to iterate through:
                container_len_uni = Selector(text=response.text).xpath('//div[@class="menu"]\
                ['+str(meta_menu_item+1)+']/div[@class="menuItems"]/div[@class="section"]['+str(n_level)+']/div\
                [@class="entryContainer"]/div[@class="'+left_or_right+'Column"]/\
                div[@class="entry"]/node()[1]//text()').extract()
#                 print "left_check:", left_or_right, "contain len:", len(container_len_uni)
            
                for section_n in range(len(container_len_uni)):                    
                    
                    #now we can get the name of that menu item...
                    menu_item_name_uni = Selector(text=response.text).xpath('//div[@class="menu"]\
                    ['+str(meta_menu_item+1)+']/div[@class="menuItems"]/div[@class="section"]\
                    ['+str(n_level)+']/div\
                    [@class="entryContainer"]/div[@class="'+left_or_right+'Column"]/div[@class="entry"]\
                    ['+str(section_n+1)+']/node()[1]//text()').extract()
                    menu_item_name     = unicodedata.normalize('NFKD', menu_item_name_uni[0]
                                                                    ).encode('ascii','ignore')
#                     print "menu_item_name:", menu_item_name
                    
                    #and then we can get the price (if there is one...)
                    try:
                        menu_item_price_uni = Selector(text=response.text).xpath('//div[@class="menu"]\
                    ['+str(meta_menu_item+1)+']/div[@class="menuItems"]/div[@class="section"]\
                    ['+str(n_level)+']/div\
                    [@class="entryContainer"]/div[@class="'+left_or_right+'Column"]/div[@class="entry"]\
                    ['+str(section_n+1)+']/node()[2]//text()').extract()
                        menu_item_price     = unicodedata.normalize('NFKD', menu_item_price_uni[0]
                                                                    ).encode('ascii','ignore')
                        menu_item_price = float(menu_item_price)
#                         print "menu_item_price:", menu_item_price
                    except:
#                         print "menu_item_price:", "price_not_available"
                        menu_item_price = 'price_not_available'
                    
                    #and finally the description (if there is one...)
                    try:
                        menu_item_desc_uni = Selector(text=response.text).xpath('//div[@class="menu"]\
                    ['+str(meta_menu_item+1)+']/div[@class="menuItems"]/div[@class="section"]\
                    ['+str(n_level)+']/div\
                    [@class="entryContainer"]/div[@class="'+left_or_right+'Column"]/div[@class="entry"]\
                    ['+str(section_n+1)+']/node()[3]//text()').extract()
                        menu_item_desc     = unicodedata.normalize('NFKD', menu_item_desc_uni[0]
                                                                    ).encode('ascii','ignore')
#                         print "menu_item_desc:", menu_item_desc
                    except:
#                         print "menu_item_desc:", "desc_not_available"
                        menu_item_desc = 'desc_not_available'

                    # Finally, I'll append my results so that when I wrap up the fuction, I can finish with
                    # a prepared set of info, dataframe ready.
                    df_ready_rows.append([venue_name,
                                       venue_desc_list,
                                       venue_menu_url,
                                       venue_rated,
                                       meta_menu_n,
                                       depth_menus_n,
                                       menu_item_name,
                                       menu_item_price,
                                       menu_item_desc])
#     df_ready_rows.append(venue_rows)
    return df_ready_rows
# 

In [10]:
len(unique_urls_from_search)
#This amounts to roughly 8% of my total venues searched i.e. 280/(70*50)

280

In [11]:
# parse_url('https://foursquare.com/v/grand-sichuan-house/4b1c3b4cf964a520bb0424e3/menu')

In [12]:
# Could be not working because they don't have a menu, or because they're not a restaurant. So could be useful to
# later determine if they are or aren't restaurants to begin with
start_time = time.time()
for menu_url in unique_urls_from_search[:30]:
    try:
        parse_url(menu_url)
    except:
        pass
    print("--- %s seconds ---" % (time.time() - start_time))

--- 0.49055480957 seconds ---
--- 12.5686228275 seconds ---
--- 12.7575688362 seconds ---
--- 12.9671459198 seconds ---
--- 13.1560468674 seconds ---
--- 13.3051497936 seconds ---
--- 25.0519609451 seconds ---
--- 25.3167629242 seconds ---
--- 25.493188858 seconds ---
--- 37.2827179432 seconds ---
--- 37.6100490093 seconds ---
--- 52.5052340031 seconds ---
--- 52.6898667812 seconds ---
--- 53.4898679256 seconds ---
--- 62.1172599792 seconds ---
--- 64.9277229309 seconds ---
--- 65.1554119587 seconds ---
--- 77.1680719852 seconds ---
--- 77.3154718876 seconds ---
--- 77.5070137978 seconds ---
--- 89.2520778179 seconds ---
--- 89.5217859745 seconds ---
--- 89.6632127762 seconds ---
--- 89.8353009224 seconds ---
--- 90.3072829247 seconds ---
--- 90.7108669281 seconds ---
--- 101.565645933 seconds ---
--- 101.976698875 seconds ---
--- 113.615211964 seconds ---
--- 113.790491819 seconds ---


In [14]:
print len(df_ready_rows)

43


In [16]:
search_else = pd.DataFrame(df_ready_rows, columns=column_headers)
search_else.shape

(43, 9)

In [17]:
for j in search_else.venue_desc_list:
    if 'vegan' in j[0]:
        print "vegans"

In [18]:
search_else.loc[:50[]

,venue_name,venue_desc_list,venue_menu_url,venue_rated,meta_menu_n,depth_menus_n,menu_item_name,menu_item_price,menu_item_desc
0,Victor's,[Mexican Restaurant],https://foursquare.com/v/victor's/4a898fc8f964...,7.5,Main Menu,Tacos,Beef Or Grilled Chicken,2.65,desc_not_available
1,Victor's,[Mexican Restaurant],https://foursquare.com/v/victor's/4a898fc8f964...,7.5,Main Menu,Tacos,Chile Verde Or Pork,2.65,desc_not_available
2,Victor's,[Mexican Restaurant],https://foursquare.com/v/victor's/4a898fc8f964...,7.5,Main Menu,Tacos,Chicken Mole,2.65,desc_not_available
3,Victor's,[Mexican Restaurant],https://foursquare.com/v/victor's/4a898fc8f964...,7.5,Main Menu,Tacos,Nachos,1.5,"Cheese, salsa, sour cream, refried beans, grou..."
4,Victor's,[Mexican Restaurant],https://foursquare.com/v/victor's/4a898fc8f964...,7.5,Main Menu,Tortas,Beef Or Grilled Chicken,5.7,desc_not_available


In [19]:
something_else.venue_menu_url.unique()

array(["https://foursquare.com/v/victor's/4a898fc8f964a520660820e3/menu",
       'https://foursquare.com/v/artesano/51e0ad55498eb7f2b6ed10e2/menu'], dtype=object)

In [ ]:
#NOTES: going to want to cull all the multiples from the menu url's that i grab?
#should i make the loop above this cell more efficient by checking and storing which url's don't work?
#B/C then I can exclude them the next time i run my loops ; BUT, it's probably better to keep track of these,
#because while some won't even be eateries, many will be eateries that simply don't have foursqare menus.
#In such cases, knowing the venue information could still be valuable, because we can surface those to users who
#wish to manually add items (could possibly add items by taking pictures of the menu where the item is located)?

In [21]:
# NOTE: I rewrote the searched_venue_ids part, to work with the .explore API endpoints
unique_venues_from_explore = []
unique_venue_names_from_explore = []
start_time = time.time()
for x, y in grid_pairs:
    try:
        explore = client.venues.explore(params={'ll': '%.2f, %.2f' % (y, x), 'llAcc':'100.0','radius': '3000',
                                               'section': 'food','limit':'50','offset':'50','sortByDistance':'1'})
        explored_venue_ids = []
        explored_venue_names = []
        for i in range(len(explore['groups'][0]['items'])):
            try:
                pulled_id = explore['groups'][0]['items'][i]['venue']['menu']['url']
                explored_venue_ids.append(pulled_id)
                pulled_name = explore['groups'][0]['items'][i]['venue']['name']
                explored_venue_names.append(pulled_name)
            except:
                pass
        for next_id, next_name in zip(explored_venue_ids, explored_venue_names):
            if 'foursquare.com' in str(next_id):
                unique_venues_from_explore.append(next_id)
                unique_venue_names_from_explore.append(next_name)
        print("--- %s seconds ---" % (time.time() - start_time))
    except:
        pass

[] []
--- 0.255792856216 seconds ---
[] []
--- 0.517818927765 seconds ---
[] []
--- 0.735308885574 seconds ---
[] []
--- 0.981924057007 seconds ---
[] []
--- 1.25415992737 seconds ---
[] []
--- 1.4582760334 seconds ---
[] []
--- 1.63622689247 seconds ---
[] []
--- 1.88748502731 seconds ---
[] []
--- 2.34382987022 seconds ---
[u'http://www.blazepizza.com/menu', u'https://foursquare.com/v/peking-restaurant/4b97141ef964a520e5f634e3/menu', u'https://foursquare.com/v/la-baguette/529d49c111d27cfebb6c8740/menu', u'https://foursquare.com/v/cocola-bakery/4b3fd429f964a520e3af25e3/menu', u'https://foursquare.com/v/little-beijing/4b1f13bff964a520b62324e3/menu', u'https://foursquare.com/v/shabu-house/4ec87aab02d5888e9d1a1da7/menu', u'https://foursquare.com/v/paradise-pizza--pasta/4af7939ef964a520eb0922e3/menu', u'https://foursquare.com/v/fuji/4a944e31f964a520262120e3/menu', u'https://foursquare.com/v/sri-thai-cuisine/4bd5111ccfa7b7132f6125da/menu', u'http://toasteatery.com/files/toastmenu_052815.pd

In [22]:
print len(unique_venue_names_from_explore), len(unique_venues_from_explore)

1362 1362


In [26]:
# Actually, through the Explore endpoint, I was able to directly grab the menu url, so no need to manually build my
# url this time...
start_time = time.time()
for menu_url in unique_venues_from_explore[:30]:
    try:
        parse_url(menu_url)
    except:
        pass
    print("--- %s seconds ---" % (time.time() - start_time))
#Takes about 4 mins for 30 url's

--- 4.78885889053 seconds ---
--- 22.3803648949 seconds ---
--- 51.3394248486 seconds ---
--- 61.2552258968 seconds ---
--- 65.0905029774 seconds ---
--- 69.841105938 seconds ---
--- 85.9875249863 seconds ---
--- 92.3588399887 seconds ---
--- 94.4333457947 seconds ---
--- 99.9304938316 seconds ---
--- 106.302963972 seconds ---
--- 117.754977942 seconds ---
--- 126.242200851 seconds ---
--- 136.669624805 seconds ---
--- 147.496289968 seconds ---
--- 151.145290852 seconds ---
--- 165.825152874 seconds ---
--- 169.614964962 seconds ---
--- 174.458040953 seconds ---
--- 176.92544198 seconds ---
--- 180.788144827 seconds ---
--- 183.491753817 seconds ---
--- 198.771272898 seconds ---
--- 203.05129981 seconds ---
--- 205.352755785 seconds ---
--- 215.284732819 seconds ---
--- 217.5648489 seconds ---
--- 229.305387974 seconds ---
--- 236.550656796 seconds ---
--- 240.609205008 seconds ---


In [36]:
explore_else = pd.DataFrame(df_ready_rows, columns=column_headers)
explore_else.shape

(3103, 9)

In [91]:
explore_else.head()

,venue_name,venue_desc_list,venue_menu_url,venue_rated,meta_menu_n,depth_menus_n,menu_item_name,menu_item_price,menu_item_desc
0,Peking Restaurant,[Chinese Restaurant],https://foursquare.com/v/peking-restaurant/4b9...,6.9,Main Menu,Appetizers,Vegetable Pot Stickers (6),4.95,desc_not_available
1,Peking Restaurant,[Chinese Restaurant],https://foursquare.com/v/peking-restaurant/4b9...,6.9,Main Menu,Appetizers,Fried Won Ton (10),3.95,desc_not_available
2,Peking Restaurant,[Chinese Restaurant],https://foursquare.com/v/peking-restaurant/4b9...,6.9,Main Menu,Appetizers,Boiled Shrimp Dumplings (10),5.5,desc_not_available
3,Peking Restaurant,[Chinese Restaurant],https://foursquare.com/v/peking-restaurant/4b9...,6.9,Main Menu,Appetizers,Combination Hot Appetizers (for Two),7.95,"2 egg rolls, 4 fried shrimps and 4 pot stickers."
4,Peking Restaurant,[Chinese Restaurant],https://foursquare.com/v/peking-restaurant/4b9...,6.9,Main Menu,Appetizers,Vegetarian Egg Rolls (4) *,3.95,desc_not_available


In [ ]:
#BELOW ARE POTENTIALLY USEFUL, BUT UNUSED MATERIAL::

In [79]:
# Here I'm just testing out a prototype search function
count = 0
loc_list = []
for i in range(explore_else.shape[0]):
    flat_desc = ' '.join(explore_else.venue_desc_list[i])
    if 'asian' in flat_desc.lower():
        count += 1
        loc_list.append(i)
print count, loc_list

155 [1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1704, 1705, 1706, 1707, 1708, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 1740, 1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844]


In [90]:
for i in loc_list:
    dffff = explore_else.loc[[i]]
dffff

,venue_name,venue_desc_list,venue_menu_url,venue_rated,meta_menu_n,depth_menus_n,menu_item_name,menu_item_price,menu_item_desc
1844,Lam Hoa Thuan Restaurant,[Asian Restaurant],https://foursquare.com/v/lam-hoa-thuan-restaur...,7.5,Main Menu,Beverages,Chedaudo,2.25,Red bean drink with coconut milk


In [72]:
# This will be the func to add the next offset to my completed venue list...
def extend_unique_venues(unique_venues, proposed_venue):
    if proposed_venue not in unique_venues:
        unique_venues.append(proposed_venue)

In [ ]:
# # Category/column titles, in order:
# # [venue_name, venue_desc_list, venue_menu_url, venue_rated], [meta_menu_n], [depth_menus_n], [menu_item_name,
# # menu_item_price, menu_item_desc]

# venue_rows = []
# for [venue_name, venue_desc_list, venue_menu_url, venue_rated] in venues: 
#     for meta_menu in meta_menu_n:
#         for depth_menu in depth_menus_n:
#             venue_rows.append([venue_name,
#                                venue_desc_list,
#                                venue_rated,
#                                meta_menu,
#                                depth_menu,
#                                menu_item_name,
#                                menu_item_price,
#                                menu_item_desc])

In [9]:
# # Category/column titles, in order:
# # [venue_name, venue_desc_list, venue_menu_url, venue_rated], [meta_menu_n], [depth_menus_n], [menu_item_name,
# # menu_item_price, menu_item_desc]

# venue_dict = {}
# for [venue_name, venue_desc_list, venue_menu_url, venue_rated] in venues:
#     venue_dict[venue_name] = {'desc_list':venue_desc_list,
#                               'menu_url':venue_menu_url,
#                               'rating':venue_rated}
    
#     for meta_menu in meta_menu_n:
#         venue_dict[venue_name][meta_menu] = {}
            
#         for depth_menu in depth_menus_n:
#             venue_dict[venue_name][meta_menu][depth_menu] = {'menu_item_name':menu_item_name,
#                                                              'menu_item_price':menu_item_price,
#                                                              'menu_item_desc':menu_item_desc}
            
